# Project 1 - Apriori
## Zack Azadian
## October 17, 2019

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#final_data = pd.read_csv("f_adult_clean_data.csv", index_col=0)

In [3]:
data = pd.read_csv("adult.data", header=None)

In [4]:
# clean data
clean_data = data

# age
for i in range(len(data)):
    clean_data.iloc[i,0] = str(clean_data.iloc[i,0])+'a'

# education number
for i in range(len(data)):
    clean_data.iloc[i,4] = str(clean_data.iloc[i,4])+'en'
    
# capital gain
for i in range(len(data)):
    clean_data.iloc[i,10] = str(clean_data.iloc[i,10])+'cg'
    
# capital loss
for i in range(len(data)):
    clean_data.iloc[i,11] = str(clean_data.iloc[i,11])+'cl'

# hours per week
for i in range(len(data)):
    clean_data.iloc[i,12] = str(clean_data.iloc[i,12])+'hw'

# replace ? with 0
clean_data = clean_data.replace(' ?',0)

# Binning
print('Binning')
for i in range(len(clean_data)):

    if clean_data.iloc[i,2] <= 50000:
        clean_data.iloc[i,2] = '0-50,000'
    elif 50000 < clean_data.iloc[i,2] <= 100000:
        clean_data.iloc[i,2] = '50,000-100,000'

    elif 100000 < clean_data.iloc[i,2] <= 150000:
        clean_data.iloc[i,2] = '100,000-150,000' 

    elif 150000 < clean_data.iloc[i,2] <= 200000:
        clean_data.iloc[i,2] = '150,000-200,000'

    elif 200000 < clean_data.iloc[i,2] <= 250000:
        clean_data.iloc[i,2] = '200,000-250,000'

    elif 250000 < clean_data.iloc[i,2] <= 300000:
        clean_data.iloc[i,2] = '250,000-300,000' 

    elif 300000 < clean_data.iloc[i,2] <= 350000:
        clean_data.iloc[i,2] = '300,000-350,000' 

    elif 350000 < clean_data.iloc[i,2] <= 400000:
        clean_data.iloc[i,2] = '350,000-400,000' 

    else:
        clean_data.iloc[i,2] = '400,000-1,484,705'
        
print('Binning complete')
        
final_data = clean_data

Binning
Binning complete


In [5]:
final_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39a,State-gov,"50,000-100,000",Bachelors,13en,Never-married,Adm-clerical,Not-in-family,White,Male,2174cg,0cl,40hw,United-States,<=50K
1,50a,Self-emp-not-inc,"50,000-100,000",Bachelors,13en,Married-civ-spouse,Exec-managerial,Husband,White,Male,0cg,0cl,13hw,United-States,<=50K
2,38a,Private,"200,000-250,000",HS-grad,9en,Divorced,Handlers-cleaners,Not-in-family,White,Male,0cg,0cl,40hw,United-States,<=50K
3,53a,Private,"200,000-250,000",11th,7en,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0cg,0cl,40hw,United-States,<=50K
4,28a,Private,"300,000-350,000",Bachelors,13en,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0cg,0cl,40hw,Cuba,<=50K


In [6]:
def build_count_c1_list(data, min_sup, z):
    d_C1 = {}
    
    for i in range(len(data)):
        for j in range(data.columns.size):
            if data.iloc[i,j] not in d_C1:
                d_C1[data.iloc[i,j]] = 1
            else:
                d_C1[data.iloc[i,j]] += 1
                
                
    min_sup_number = len(data)*min_sup
    keys_list = list(d_C1.keys())
    
    # apply minsup
    d_C1_f = {}
    for i in keys_list:
        if int(d_C1[i]) >= min_sup_number:
            d_C1_f[i] = d_C1[i]
    print("Frequency set length {} with frequency:".format(z),d_C1_f)
    return d_C1_f

In [7]:
def apply_min_sup(d, sample, min_sup, z):

    min_sup_number = len(sample)*min_sup
    
    d_lk = {}
    keys_list = list(d.keys())
    
    for i in keys_list:
        if int(d[i]) >= min_sup_number:
            d_lk[i] = d[i]
    print("Frequency set length {} with frequency:".format(z),d_lk)
    return d_lk   

In [8]:
def cross_join(key1, key2):
    tup = []
    if type(key1) == str:
        tup.append(key1)
        tup.append(key2)
    else:
        for i,j in zip(key1, key2):
            if i == j:
                tup.append(i)
            elif i < j:
                tup.append(i)
                tup.append(j)
                
    if len(tup) > (len(key1)+1):
         return None
    else:
         return list(tup)

In [9]:
def has_infreq_subset(can, d):
    count = 0
    
    if len(can)==2:
        return False 
        
    else:
        for i in range(len(can)):
            for j in range(i+1, len(can)):
                subsets=[]
                subsets.append(can[i])
                subsets.append(can[j])

                if tuple(subsets) in d.keys():
                    count += 1
            
            #print(subsets)
            
        return count != len(can)

In [10]:
def candidate_keys(Lk, z):
    
    ck = []
    key_list = list(Lk.keys())
    
    if key_list[0] is tuple:
        for i in range(len(key_list)):
            key_list[i] = list(key_list[i])
    else:
        key_list.sort()

        for i in range(len(key_list)): #0-5
            for j in range(i+1, len(key_list)): #1-5
                can = cross_join(key_list[i], key_list[j])
                
                if can is not None:
                    if has_infreq_subset(can, Lk) == False:
                        if can not in ck:
                            
                            if len(can)== len(key_list[0])/len(key_list[0]) + z:
                                
                            #if len(can)== len(key_list[0])/len(key_list[0]) +1:
                                
                                
                                ck.append(can) 
                                
    return ck

In [11]:
def tup_scan(cankeys_aslists, dataframe):

    d_ck = {}
    
    for i in range(len(cankeys_aslists)):
        scan_count = 0
        x = cankeys_aslists[i]
        
        for k in range(len(dataframe)):
            inrow_count = 0
            
            for j in range(len(x)):
                
                for l in range(dataframe.columns.size):
                    if x[j] == dataframe.iloc[k,l]:
                        inrow_count += 1
                        
            if inrow_count == len(x):
                scan_count += 1
                
            d_ck[tuple(x)] = scan_count 
                    
    return d_ck

In [12]:
def z_apriori(dataframe, min_sup):
    
    z = 1
    
    Lk = build_count_c1_list(dataframe, min_sup, z)
        
    while len(Lk) > 0:
        Ck_candidate = candidate_keys(Lk, z)
        Ck = tup_scan(Ck_candidate, dataframe)
        Lk = apply_min_sup(Ck, dataframe, min_sup, z+1)
        z+=1 

In [26]:
z_apriori(final_data, 0.7)

Frequency set length 1 with frequency: {' White': 27816, '0cl': 31042, ' United-States': 29170, ' <=50K': 24720, '0cg': 29849}
Frequency set length 2 with frequency: {(' <=50K', '0cg'): 23685, (' <=50K', '0cl'): 23974, (' United-States', ' White'): 25621, (' United-States', '0cg'): 26699, (' United-States', '0cl'): 27791, (' White', '0cg'): 25407, (' White', '0cl'): 26470, ('0cg', '0cl'): 28330}
Frequency set length 3 with frequency: {(' <=50K', '0cg', '0cl'): 22939, (' United-States', ' White', '0cg'): 23369, (' United-States', ' White', '0cl'): 24349, (' United-States', '0cg', '0cl'): 25320, (' White', '0cg', '0cl'): 24061}
Frequency set length 4 with frequency: {}


## Improvement

In [14]:
def Z_build_count_c1_list(data, min_sup, z):
    d_C1 = {}
    
    for i in range(len(data)):
        for j in range(data.columns.size):
            
            if data.iloc[i,j] not in d_C1:
                d_C1[data.iloc[i,j]] = 1
    
            else:
                d_C1[data.iloc[i,j]] += 1
                
    min_sup_number = len(data)*min_sup
    keys_list = list(d_C1.keys())
    
    # apply minsup
    d_C1_f = {}
    for i in keys_list:
        if int(d_C1[i]) >= min_sup_number:
            d_C1_f[i] = d_C1[i]
            
    print("Frequency set length {} with frequency:".format(z),d_C1_f)
    
    return d_C1_f

In [15]:
def Z_apply_min_sup(d, sample, min_sup, z):

    min_sup_number = len(sample)*min_sup
    
    d_lk = {}
    keys_list = list(d.keys())
    
    for i in keys_list:
        if int(d[i]) >= min_sup_number:
            d_lk[i] = d[i]
    print("Frequency set length {} with frequency:".format(z),d_lk)
    return d_lk   

In [16]:
def improved_z_apriori(dataframe, min_sup):
    
    def Z_tup_scan(cankeys_aslists, dataframe):

        d_ck = {}
    
        for i in range(len(cankeys_aslists)):
            scan_count = 0
            x = cankeys_aslists[i]

            for k in row_dict.keys():
                inrow_count = 0

                for j in range(len(x)):
                    for l in range(dataframe.columns.size):
                        if x[j] == dataframe.iloc[k,l]:
                            inrow_count += 1
                            temp_row_dict[k]=None

                if inrow_count == len(x):
                    scan_count += 1

            d_ck[tuple(x)] = scan_count 

        return d_ck
    
    z = 1
    
    Lk = Z_build_count_c1_list(dataframe, min_sup, z)
    
    temp_row_dict = {}
    row_dict = {}
    for i in range(len(final_data)):
        row_dict[i]=None
    
    while len(Lk) > 0:
        Ck_candidate = candidate_keys(Lk, z)
        Ck = Z_tup_scan(Ck_candidate, dataframe)
        row_dict = temp_row_dict
        Lk = Z_apply_min_sup(Ck, dataframe, min_sup, z+1)
        z+=1 

In [17]:
import time

In [18]:
# original
start = time.time()

z_apriori(final_data, 0.8)

end = time.time()

time_taken = end - start
print('Time: ',time_taken)

Frequency set length 1 with frequency: {' White': 27816, '0cl': 31042, ' United-States': 29170, '0cg': 29849}
Frequency set length 2 with frequency: {(' United-States', '0cg'): 26699, (' United-States', '0cl'): 27791, (' White', '0cl'): 26470, ('0cg', '0cl'): 28330}
Frequency set length 3 with frequency: {}
Time:  63.763564109802246


In [19]:
# improved
start = time.time()

improved_z_apriori(final_data, 0.8)

end = time.time()

time_taken = end - start
print('Time: ',time_taken)

Frequency set length 1 with frequency: {' White': 27816, '0cl': 31042, ' United-States': 29170, '0cg': 29849}
Frequency set length 2 with frequency: {(' United-States', '0cg'): 26699, (' United-States', '0cl'): 27791, (' White', '0cl'): 26470, ('0cg', '0cl'): 28330}
Frequency set length 3 with frequency: {}
Time:  61.542356967926025


In [20]:
start = time.time()

z_apriori(final_data, 0.7)

end = time.time()

time_taken = end - start
print('Time: ',time_taken)

Frequency set length 1 with frequency: {' White': 27816, '0cl': 31042, ' United-States': 29170, ' <=50K': 24720, '0cg': 29849}
Frequency set length 2 with frequency: {(' <=50K', '0cg'): 23685, (' <=50K', '0cl'): 23974, (' United-States', ' White'): 25621, (' United-States', '0cg'): 26699, (' United-States', '0cl'): 27791, (' White', '0cg'): 25407, (' White', '0cl'): 26470, ('0cg', '0cl'): 28330}
Frequency set length 3 with frequency: {(' <=50K', '0cg', '0cl'): 22939, (' United-States', ' White', '0cg'): 23369, (' United-States', ' White', '0cl'): 24349, (' United-States', '0cg', '0cl'): 25320, (' White', '0cg', '0cl'): 24061}
Frequency set length 4 with frequency: {}
Time:  137.07891011238098


In [21]:
start = time.time()

improved_z_apriori(final_data, 0.7)

end = time.time()

time_taken = end - start
print('Time: ',time_taken)

Frequency set length 1 with frequency: {' White': 27816, '0cl': 31042, ' United-States': 29170, ' <=50K': 24720, '0cg': 29849}
Frequency set length 2 with frequency: {(' <=50K', '0cg'): 23685, (' <=50K', '0cl'): 23974, (' United-States', ' White'): 25621, (' United-States', '0cg'): 26699, (' United-States', '0cl'): 27791, (' White', '0cg'): 25407, (' White', '0cl'): 26470, ('0cg', '0cl'): 28330}
Frequency set length 3 with frequency: {(' <=50K', '0cg', '0cl'): 22939, (' United-States', ' White', '0cg'): 23369, (' United-States', ' White', '0cl'): 24349, (' United-States', '0cg', '0cl'): 25320, (' White', '0cg', '0cl'): 24061}
Frequency set length 4 with frequency: {}
Time:  131.16005992889404


In [22]:
# randomly sampling 5000 rows from Adult Dataframe to run timing tests
data5k = final_data.sample(n=5000)

In [23]:
start = time.time()

z_apriori(data5k, 0.9)

end = time.time()

time_taken = end - start
print('Time: ',time_taken)

Frequency set length 1 with frequency: {'0cg': 4581, '0cl': 4754, ' United-States': 4516}
Frequency set length 2 with frequency: {}
Time:  4.627982139587402
